In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.2.0'

Data Preprocessing

In [3]:
# Importing Dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

In [4]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [5]:
print(y)

[1 0 1 ... 1 1 0]


In [6]:
# Encoding Categorical Data

# LabelEncoding the "Gender" feature
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [7]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [8]:
# OneHotEncoding the "Geography" feature
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder='passthrough')
# [1] : represents "Geography" column
X = np.array(ct.fit_transform(X))

In [9]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Splitting the dataset into Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

Feature Scaling

In [11]:
# In ANN we need to apply feature scalling in the dummy variables as well
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Building ANN

In [12]:
# Initializing ANN
ann = tf.keras.models.Sequential()

In [13]:
# Adding input layer and first hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu', ))

# While adding the 1st hidden layer, the input layer is automatically initiated where number of input neurons is equal to number of features
# Number of neurons in the 1st hudden layer is 6
# We are using Dense class as we want fully connected layer

In [14]:
# Adding 2nd hidden layer
ann.add(tf.keras.layers.Dense(units=6, activation='relu', ))

In [15]:
# Adding Output layer
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid', ))

# Since there are 2 class we want only one output neuron and sigmoid activation function. 
# "Sigmoid" activation function not only gives us the predictions but the probabilities that the binary outcome is 1.
# For non-binary classifications (> 2 class), we use "softmax" instead of "sigmoid" as our activation function. 

Training ANN

In [16]:
# Compiling ANN
ann.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])

# We choose the "adam" optimizer as the best one that can perform Stochastic Gradient Descent.
# The optimizer updates the weights in order to reduce the loss error between the predictions and the actual results
# The "loss" function is a way to compute the difference between the predictions and the real results.
# When we are doing binary classification, the loss function should always be binary_crossentropy. 
# For non-binary classifications (> 2 class), we replace with "categorical_crossentropy".

In [17]:
# Training ANN on training set
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

# batch_size = 32 is the default size

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.7095 - accuracy: 0.5855
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5352 - accuracy: 0.7738
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4905 - accuracy: 0.7954
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4639 - accuracy: 0.7976
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4467 - accuracy: 0.8029
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4358 - accuracy: 0.8074
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4284 - accuracy: 0.8125
Epoch 8/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4227 - accuracy: 0.8146
Epoch 9/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4173 - accuracy: 0.8196
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4122 - accura

Making the predictions and evaluating the model

In [18]:
# Predicting Test set result 
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [19]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1505   90]
 [ 192  213]]


0.859

**Task:** 

Use ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 yrs old

Tenure: 3 yrs

Balance: 60000

Number of Products: 2

Credit card ?: Yes

Active Member? : Yes

Estimated Salary: 50000

**Will this customer leave?**

In [20]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

# Format accepted by the predict() method is 2D array
# We need to scale down our values before we can use it as input
# We can change the value of threshold i.e 0.5 according to the requirements 

[[False]]
